In [1]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt

# hyper parameters
learning_rate = 0.001
training_epochs = 3000
batch_size = 132

# input place holders
training_data = np.load('train_data.npy')
testing_data = np.load('test_data.npy')

x_data_test = testing_data[:, 0:-1]
y_data_test = testing_data[:, [-1]]

x_data = training_data[:, 0:-1]
y_data = training_data[:, [-1]]

x_data = x_data.reshape(-1,64,64,3)
x_data_test = x_data_test.reshape(-1,64,64,3)

X = tf.placeholder(tf.float32, [None,64,64,3])
Y = tf.placeholder(tf.int32, [None, 1])

Y_one_hot = tf.one_hot(Y, 2)  # one hot
Y_one_hot = tf.reshape(Y_one_hot, [-1, 2])

# L1 ImgIn shape=(?, 128, 128, 3)
W1 = tf.Variable(tf.random_normal([3, 3, 3, 64], stddev=0.01))
#    Conv     -> (?, 64, 64, 128)
#    Pool     -> (?, 64, 64, 128)
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
# L1 = tf.nn.dropout(L1, 0.7)

# L2 ImgIn shape=(?, 64, 64, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 64, 32], stddev=0.01))
#    Conv      ->(?, 64, 64, 64)
#    Pool      ->(?, 32, 32, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.nn.dropout(L2, 0.3)

# # L3 
# W3 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
# #    Conv      ->(?, 32, 32, 64)
# #    Pool      ->(?, 16, 16, 64)
# L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
# L3 = tf.nn.relu(L3)
# L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1],
#                     strides=[1, 2, 2, 1], padding='SAME')
# L3 = tf.nn.dropout(L3, 0.3)

#L4
W4 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 16, 16, 64)
#    Pool      ->(?, 8, 8, 64)
L4 = tf.nn.conv2d(L2, W4, strides=[1, 1, 1, 1], padding='SAME')
L4 = tf.nn.relu(L4)
L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L4_flat = tf.reshape(L4, [-1, 8* 8 * 64])


# Final FC 4x4x64 inputs -> 2 outputs
W5 = tf.get_variable("W5", shape=[8 * 8 * 64, 1024],
                     initializer=tf.contrib.layers.xavier_initializer())
# B1 = tf.Variable(tf.random_normal([None,1024]))
# logits = tf.matmul(L4_flat, W5) + b
L5 = tf.nn.relu(tf.matmul(L4_flat, W5))

W6 = tf.get_variable("W6", shape=[1024, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
# B2 = tf.Variable(tf.random_normal([None,1024]))
L6 = tf.nn.relu(tf.matmul(L5, W6))

W7 = tf.get_variable("W7", shape=[512, 2],
                     initializer=tf.contrib.layers.xavier_initializer())
# B3 = tf.Variable(tf.random_normal([None,512]))
L7 = tf.nn.relu(tf.matmul(L6, W7))

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=L7, labels=Y_one_hot))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = zip(range(0,len(x_data),batch_size),
                      range(batch_size,len(x_data)+1,batch_size))
    for start,end in total_batch:
        feed_dict = {X: x_data[start:end], Y: y_data[start:end]}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / len(list(zip(range(0,len(x_data),batch_size),
                      range(batch_size,len(x_data)+1,batch_size))))

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(L7, 1), tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: x_data_test, Y: y_data_test}))

# # Get one and predict
# r = random.randint(0, 10)
# print("Label: ", y_data_test[r])
# print("Prediction: ", sess.run(
#     tf.argmax(logits, 1), feed_dict={X: x_data_test[r]}))

# plt.imshow(mnist.test.images[r:r + 1].
#           reshape(28, 28), cmap='Greys', interpolation='nearest')
# plt.show()

W0911 12:36:28.955671 20108 deprecation.py:506] From <ipython-input-1-30642a78a068>:48: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0911 12:36:28.959660 20108 nn_ops.py:4224] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
W0911 12:36:29.970671 20108 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0911 12:36:30.005578 20108 deprecation.py:323] From <ipython-input-1-30642a78a06

Learning started. It takes sometime.
Epoch: 0001 cost = 9.772026867
Epoch: 0002 cost = 0.348351104
Epoch: 0003 cost = 0.690723896
Epoch: 0004 cost = 0.693147182
Epoch: 0005 cost = 0.693147182
Epoch: 0006 cost = 0.693147182
Epoch: 0007 cost = 0.693147182
Epoch: 0008 cost = 0.693147182
Epoch: 0009 cost = 0.693147182
Epoch: 0010 cost = 0.693147182
Epoch: 0011 cost = 0.693147182
Epoch: 0012 cost = 0.693147182
Epoch: 0013 cost = 0.693147182
Epoch: 0014 cost = 0.693147182
Epoch: 0015 cost = 0.693147182
Epoch: 0016 cost = 0.693147182
Epoch: 0017 cost = 0.693147182
Epoch: 0018 cost = 0.693147182
Epoch: 0019 cost = 0.693147182
Epoch: 0020 cost = 0.693147182
Epoch: 0021 cost = 0.693147182
Epoch: 0022 cost = 0.693147182
Epoch: 0023 cost = 0.693147182
Epoch: 0024 cost = 0.693147182
Epoch: 0025 cost = 0.693147182
Epoch: 0026 cost = 0.693147182
Epoch: 0027 cost = 0.693147182
Epoch: 0028 cost = 0.693147182
Epoch: 0029 cost = 0.693147182
Epoch: 0030 cost = 0.693147182
Epoch: 0031 cost = 0.693147182
Ep

Epoch: 0265 cost = 0.693147182
Epoch: 0266 cost = 0.693147182
Epoch: 0267 cost = 0.693147182
Epoch: 0268 cost = 0.693147182
Epoch: 0269 cost = 0.693147182
Epoch: 0270 cost = 0.693147182
Epoch: 0271 cost = 0.693147182
Epoch: 0272 cost = 0.693147182
Epoch: 0273 cost = 0.693147182
Epoch: 0274 cost = 0.693147182
Epoch: 0275 cost = 0.693147182
Epoch: 0276 cost = 0.693147182
Epoch: 0277 cost = 0.693147182
Epoch: 0278 cost = 0.693147182
Epoch: 0279 cost = 0.693147182
Epoch: 0280 cost = 0.693147182
Epoch: 0281 cost = 0.693147182
Epoch: 0282 cost = 0.693147182
Epoch: 0283 cost = 0.693147182
Epoch: 0284 cost = 0.693147182
Epoch: 0285 cost = 0.693147182
Epoch: 0286 cost = 0.693147182
Epoch: 0287 cost = 0.693147182
Epoch: 0288 cost = 0.693147182
Epoch: 0289 cost = 0.693147182
Epoch: 0290 cost = 0.693147182
Epoch: 0291 cost = 0.693147182
Epoch: 0292 cost = 0.693147182
Epoch: 0293 cost = 0.693147182
Epoch: 0294 cost = 0.693147182
Epoch: 0295 cost = 0.693147182
Epoch: 0296 cost = 0.693147182
Epoch: 0

Epoch: 0530 cost = 0.693147182
Epoch: 0531 cost = 0.693147182
Epoch: 0532 cost = 0.693147182
Epoch: 0533 cost = 0.693147182
Epoch: 0534 cost = 0.693147182
Epoch: 0535 cost = 0.693147182
Epoch: 0536 cost = 0.693147182
Epoch: 0537 cost = 0.693147182
Epoch: 0538 cost = 0.693147182
Epoch: 0539 cost = 0.693147182
Epoch: 0540 cost = 0.693147182
Epoch: 0541 cost = 0.693147182
Epoch: 0542 cost = 0.693147182
Epoch: 0543 cost = 0.693147182
Epoch: 0544 cost = 0.693147182
Epoch: 0545 cost = 0.693147182
Epoch: 0546 cost = 0.693147182
Epoch: 0547 cost = 0.693147182
Epoch: 0548 cost = 0.693147182
Epoch: 0549 cost = 0.693147182
Epoch: 0550 cost = 0.693147182
Epoch: 0551 cost = 0.693147182
Epoch: 0552 cost = 0.693147182
Epoch: 0553 cost = 0.693147182
Epoch: 0554 cost = 0.693147182
Epoch: 0555 cost = 0.693147182
Epoch: 0556 cost = 0.693147182
Epoch: 0557 cost = 0.693147182
Epoch: 0558 cost = 0.693147182
Epoch: 0559 cost = 0.693147182
Epoch: 0560 cost = 0.693147182
Epoch: 0561 cost = 0.693147182
Epoch: 0

Epoch: 0795 cost = 0.693147182
Epoch: 0796 cost = 0.693147182
Epoch: 0797 cost = 0.693147182
Epoch: 0798 cost = 0.693147182
Epoch: 0799 cost = 0.693147182
Epoch: 0800 cost = 0.693147182
Epoch: 0801 cost = 0.693147182
Epoch: 0802 cost = 0.693147182
Epoch: 0803 cost = 0.693147182
Epoch: 0804 cost = 0.693147182
Epoch: 0805 cost = 0.693147182
Epoch: 0806 cost = 0.693147182
Epoch: 0807 cost = 0.693147182
Epoch: 0808 cost = 0.693147182
Epoch: 0809 cost = 0.693147182
Epoch: 0810 cost = 0.693147182
Epoch: 0811 cost = 0.693147182
Epoch: 0812 cost = 0.693147182
Epoch: 0813 cost = 0.693147182
Epoch: 0814 cost = 0.693147182
Epoch: 0815 cost = 0.693147182
Epoch: 0816 cost = 0.693147182
Epoch: 0817 cost = 0.693147182
Epoch: 0818 cost = 0.693147182
Epoch: 0819 cost = 0.693147182
Epoch: 0820 cost = 0.693147182
Epoch: 0821 cost = 0.693147182
Epoch: 0822 cost = 0.693147182
Epoch: 0823 cost = 0.693147182
Epoch: 0824 cost = 0.693147182
Epoch: 0825 cost = 0.693147182
Epoch: 0826 cost = 0.693147182
Epoch: 0

...

Epoch: 1859 cost = 0.693147182
Epoch: 1860 cost = 0.693147182
Epoch: 1861 cost = 0.693147182
Epoch: 1862 cost = 0.693147182
Epoch: 1863 cost = 0.693147182
Epoch: 1864 cost = 0.693147182
Epoch: 1865 cost = 0.693147182
Epoch: 1866 cost = 0.693147182
Epoch: 1867 cost = 0.693147182
Epoch: 1868 cost = 0.693147182
Epoch: 1869 cost = 0.693147182
Epoch: 1870 cost = 0.693147182
Epoch: 1871 cost = 0.693147182
Epoch: 1872 cost = 0.693147182
Epoch: 1873 cost = 0.693147182
Epoch: 1874 cost = 0.693147182
Epoch: 1875 cost = 0.693147182
Epoch: 1876 cost = 0.693147182
Epoch: 1877 cost = 0.693147182
Epoch: 1878 cost = 0.693147182
Epoch: 1879 cost = 0.693147182
Epoch: 1880 cost = 0.693147182
Epoch: 1881 cost = 0.693147182
Epoch: 1882 cost = 0.693147182
Epoch: 1883 cost = 0.693147182
Epoch: 1884 cost = 0.693147182
Epoch: 1885 cost = 0.693147182
Epoch: 1886 cost = 0.693147182
Epoch: 1887 cost = 0.693147182
Epoch: 1888 cost = 0.693147182
Epoch: 1889 cost = 0.693147182
Epoch: 1890 cost = 0.693147182
Epoch: 1

Epoch: 2125 cost = 0.693147182
Epoch: 2126 cost = 0.693147182
Epoch: 2127 cost = 0.693147182
Epoch: 2128 cost = 0.693147182
Epoch: 2129 cost = 0.693147182
Epoch: 2130 cost = 0.693147182
Epoch: 2131 cost = 0.693147182
Epoch: 2132 cost = 0.693147182
Epoch: 2133 cost = 0.693147182
Epoch: 2134 cost = 0.693147182
Epoch: 2135 cost = 0.693147182
Epoch: 2136 cost = 0.693147182
Epoch: 2137 cost = 0.693147182
Epoch: 2138 cost = 0.693147182
Epoch: 2139 cost = 0.693147182
Epoch: 2140 cost = 0.693147182
Epoch: 2141 cost = 0.693147182
Epoch: 2142 cost = 0.693147182
Epoch: 2143 cost = 0.693147182
Epoch: 2144 cost = 0.693147182
Epoch: 2145 cost = 0.693147182
Epoch: 2146 cost = 0.693147182
Epoch: 2147 cost = 0.693147182
Epoch: 2148 cost = 0.693147182
Epoch: 2149 cost = 0.693147182
Epoch: 2150 cost = 0.693147182
Epoch: 2151 cost = 0.693147182
Epoch: 2152 cost = 0.693147182
Epoch: 2153 cost = 0.693147182
Epoch: 2154 cost = 0.693147182
Epoch: 2155 cost = 0.693147182
Epoch: 2156 cost = 0.693147182
Epoch: 2

Epoch: 2390 cost = 0.693147182
Epoch: 2391 cost = 0.693147182
Epoch: 2392 cost = 0.693147182
Epoch: 2393 cost = 0.693147182
Epoch: 2394 cost = 0.693147182
Epoch: 2395 cost = 0.693147182
Epoch: 2396 cost = 0.693147182
Epoch: 2397 cost = 0.693147182
Epoch: 2398 cost = 0.693147182
Epoch: 2399 cost = 0.693147182
Epoch: 2400 cost = 0.693147182
Epoch: 2401 cost = 0.693147182
Epoch: 2402 cost = 0.693147182
Epoch: 2403 cost = 0.693147182
Epoch: 2404 cost = 0.693147182
Epoch: 2405 cost = 0.693147182
Epoch: 2406 cost = 0.693147182
Epoch: 2407 cost = 0.693147182
Epoch: 2408 cost = 0.693147182
Epoch: 2409 cost = 0.693147182
Epoch: 2410 cost = 0.693147182
Epoch: 2411 cost = 0.693147182
Epoch: 2412 cost = 0.693147182
Epoch: 2413 cost = 0.693147182
Epoch: 2414 cost = 0.693147182
Epoch: 2415 cost = 0.693147182
Epoch: 2416 cost = 0.693147182
Epoch: 2417 cost = 0.693147182
Epoch: 2418 cost = 0.693147182
Epoch: 2419 cost = 0.693147182
Epoch: 2420 cost = 0.693147182
Epoch: 2421 cost = 0.693147182
Epoch: 2

Epoch: 2656 cost = 0.693147182
Epoch: 2657 cost = 0.693147182
Epoch: 2658 cost = 0.693147182
Epoch: 2659 cost = 0.693147182
Epoch: 2660 cost = 0.693147182
Epoch: 2661 cost = 0.693147182
Epoch: 2662 cost = 0.693147182
Epoch: 2663 cost = 0.693147182
Epoch: 2664 cost = 0.693147182
Epoch: 2665 cost = 0.693147182
Epoch: 2666 cost = 0.693147182
Epoch: 2667 cost = 0.693147182
Epoch: 2668 cost = 0.693147182
Epoch: 2669 cost = 0.693147182
Epoch: 2670 cost = 0.693147182
Epoch: 2671 cost = 0.693147182
Epoch: 2672 cost = 0.693147182
Epoch: 2673 cost = 0.693147182
Epoch: 2674 cost = 0.693147182
Epoch: 2675 cost = 0.693147182
Epoch: 2676 cost = 0.693147182
Epoch: 2677 cost = 0.693147182
Epoch: 2678 cost = 0.693147182
Epoch: 2679 cost = 0.693147182
Epoch: 2680 cost = 0.693147182
Epoch: 2681 cost = 0.693147182
Epoch: 2682 cost = 0.693147182
Epoch: 2683 cost = 0.693147182
Epoch: 2684 cost = 0.693147182
Epoch: 2685 cost = 0.693147182
Epoch: 2686 cost = 0.693147182
Epoch: 2687 cost = 0.693147182
Epoch: 2

Epoch: 2921 cost = 0.693147182
Epoch: 2922 cost = 0.693147182
Epoch: 2923 cost = 0.693147182
Epoch: 2924 cost = 0.693147182
Epoch: 2925 cost = 0.693147182
Epoch: 2926 cost = 0.693147182
Epoch: 2927 cost = 0.693147182
Epoch: 2928 cost = 0.693147182
Epoch: 2929 cost = 0.693147182
Epoch: 2930 cost = 0.693147182
Epoch: 2931 cost = 0.693147182
Epoch: 2932 cost = 0.693147182
Epoch: 2933 cost = 0.693147182
Epoch: 2934 cost = 0.693147182
Epoch: 2935 cost = 0.693147182
Epoch: 2936 cost = 0.693147182
Epoch: 2937 cost = 0.693147182
Epoch: 2938 cost = 0.693147182
Epoch: 2939 cost = 0.693147182
Epoch: 2940 cost = 0.693147182
Epoch: 2941 cost = 0.693147182
Epoch: 2942 cost = 0.693147182
Epoch: 2943 cost = 0.693147182
Epoch: 2944 cost = 0.693147182
Epoch: 2945 cost = 0.693147182
Epoch: 2946 cost = 0.693147182
Epoch: 2947 cost = 0.693147182
Epoch: 2948 cost = 0.693147182
Epoch: 2949 cost = 0.693147182
Epoch: 2950 cost = 0.693147182
Epoch: 2951 cost = 0.693147182
Epoch: 2952 cost = 0.693147182
Epoch: 2

In [30]:
print("Label: ", y_data)
print("Prediction: ", sess.run(
    tf.argmax(L7, 1), feed_dict={X: x_data}))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: x_data, Y: y_data}))

Label:  [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [

In [31]:
print("Label: ", y_data_test)
print("Prediction: ", sess.run(
    tf.argmax(L7, 1), feed_dict={X: x_data_test}))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: x_data_test, Y: y_data_test}))

Label:  [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
Prediction:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.52


In [27]:
correct_prediction = tf.equal(tf.argmax(L7, 1), tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: x_data_test, Y: y_data_test}))


Accuracy: 0.52
